# Wrangling DataFrames with Pandas    
## Author: Erika Duan    

![](../02_figures/04_manipulating-pandas-header.jpg)

# Dataset background  

We will be using open government data to familiarise ourselves with Pandas DataFrame manipulations. The dataset we will use describes the [index of disadvantage by schools in South Australia](https://data.gov.au/dataset/ds-sa-4ab8a539-eab5-48e2-8d88-a599d6114126/details?q=school), with a focus on [2020 data](https://data.gov.au/dataset/ds-sa-4ab8a539-eab5-48e2-8d88-a599d6114126/distribution/dist-sa-86d35483-feff-42b5-ac05-ad3186ac39de/details?q=school).     

+ The index is a socio-economic index, used by the South Australian Department for Education to allocate resources to schools to address educational disadvantage related to socio-economic status.  
+ The most disadvantaged schools have an index of 1.    
+ The least disadvantaged have an index of 7.   
+ Data is currently available for 2017, 2018 and 2020.  

# Loading data     

We can load data in two different ways:  

1. Locally using the `pathlib` library and the Pandas `read_csv` function.  
2. Directly through the data.gov.au [CKAN data API](https://docs.ckan.org/en/latest/maintaining/datastore.html) as documented [here]().    

**Note:** A guide to querying open data via CKAN can be found [here](https://alan-y.netlify.app/post/ckan-api/#querying-ckan).  

In [1]:
#-----method 1: load csv file locally-----  
import pathlib
import pandas as pd     

working_dir = pathlib.Path.cwd() # get current working directory  
data_path = working_dir.parent / '01_raw_data' / '04_index-of-disadvantage-by-school-2020.csv'

raw_data = pd.read_csv(data_path)  
raw_data.head()

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
3,4,997,Adelaide East Education Centre,Paradise,5075,Special Education,6,-34.873042,138.672971
4,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678


In [2]:
#-----method 2: access data as JSON file through CKAN data API-----  
import pathlib
import numpy as np
import pandas as pd
import requests
import json

url = "https://data.sa.gov.au/data/api/3/action/datastore_search?resource_id=86d35483-feff-42b5-ac05-ad3186ac39de&limit=512"
json_content = requests.get(url).json() # returns a dictionary object 

json_content # checks json_query

{'help': 'https://data.sa.gov.au/data/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 512,
  'records_format': 'objects',
  'resource_id': '86d35483-feff-42b5-ac05-ad3186ac39de',
  'total_estimation_threshold': None,
  'records': [{'_id': 1,
    'ID': 1,
    'School_Number': 536,
    'School_Name': 'Aberfoyle Hub R-7 School',
    'Suburb': 'Aberfoyle Park',
    'Post_Code': 5159,
    'Type_of_Schooling': 'Primary Education',
    'Index_of_Educational_Disadvantage': 6,
    'Latitude': -35.078139,
    'Longitude': 138.595142},
   {'_id': 2,
    'ID': 2,
    'School_Number': 1673,
    'School_Name': 'Aberfoyle Park High School',
    'Suburb': 'Aberfoyle Park',
    'Post_Code': 5159,
    'Type_of_Schooling': 'Secondary Education',
    'Index_of_Educational_Disadvantage': 6,
    'Latitude': -35.074007,
    'Longitude': 138.594181},
   {'_id': 3,
    'ID': 3,
    'School_Number': 1919,
    'School_Name': 'Adelaide Botanic High Sc

In [3]:
#-----convert json file to Pandas DataFrame-----
content = json.dumps(json_content, indent = 1, sort_keys = False) # JSON encoder returns a JSON file format/ string object 

df = pd.read_json(content)
df = pd.json_normalize(df['result']['records']) # normalize semi-structured JSON data into a flat table

df.head()

,_id,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
3,4,4,997,Adelaide East Education Centre,Paradise,5075,Special Education,6,-34.873042,138.672971
4,5,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678


**Note:** An introduction to the JSON file format can be found [here](https://towardsdatascience.com/an-introduction-to-json-c9acb464f43e).  

# Examining DataFrame structure  

We can examine the properties of our DataFrame using the following functions and methods:  
+ `df.shape()` to return the DataFrame dimensions.     
+ `df.info()` to return a summary of all DataFrame columns, missing values and column object types.    
+ `df.columns` to return an index of all column names.  
+ `df.describe()` to return a simple statistical summary of each column's values.  

**Note:** You can also configure Jupyter notebook to display all columns with `pd.set_option("display.max.columns", None)`. 

In [4]:
#-----example 1.1-----  
raw_data.shape

(512, 9)

In [5]:
#-----example 1.2----- 
print("The raw data has {} rows and {} columns.".format(raw_data.shape[0], raw_data.shape[1]))

The raw data has 512 rows and 9 columns.


In [6]:
#-----example 2-----  
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 512 non-null    int64  
 1   School_Number                      512 non-null    int64  
 2   School_Name                        512 non-null    object 
 3   Suburb                             512 non-null    object 
 4   Post_Code                          512 non-null    int64  
 5   Type_of_Schooling                  512 non-null    object 
 6   Index_of_Educational_Disadvantage  512 non-null    int64  
 7   Latitude                           512 non-null    float64
 8   Longitude                          512 non-null    float64
dtypes: float64(2), int64(4), object(3)
memory usage: 36.1+ KB


In [7]:
#-----example 3-----  
raw_data.columns

Index(['ID', 'School_Number', 'School_Name', 'Suburb', 'Post_Code',
       'Type_of_Schooling', 'Index_of_Educational_Disadvantage', 'Latitude',
       'Longitude'],
      dtype='object')

In [8]:
#-----example 4-----  
raw_data.describe() # only operates on columns containing integers or floats  

,ID,School_Number,Post_Code,Index_of_Educational_Disadvantage,Latitude,Longitude
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,256.500000,771.277344,5167.619141,4.304688,-34.592120,138.502559
std,147.945936,451.007245,608.153567,1.868719,1.633955,1.529273
min,1.000000,104.000000,872.000000,1.000000,-38.002525,129.172118
25%,128.750000,388.500000,5085.750000,3.000000,-35.029207,138.505453
50%,256.500000,754.500000,5163.000000,5.000000,-34.856216,138.621977
75%,384.250000,1017.250000,5351.000000,6.000000,-34.440930,138.768058
max,512.000000,1919.000000,5734.000000,7.000000,-26.151709,140.956465


# Subsetting DataFrame rows and columns   

## Subsetting DataFrame rows  

As [previously covered](https://github.com/erikaduan/Introduction-to-Python/blob/master/03_notebooks/03_Python-data-structures.ipynb), subsetting DataFrame rows can be approached in two different ways:   

+ Using `loc` i.e. via the explicit labels-based index.  
+ Using `iloc` i.e. via the implicit 0-based index.  

In [9]:
#-----example 1----- 
raw_data.iloc[0:2+1] # subset the first 3 rows    

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [10]:
#-----example 2-----  
raw_data.iloc[0:2+1, 1] # returns the first 3 rows from the second column    

0     536
1    1673
2    1919
Name: School_Number, dtype: int64

In [11]:
#-----example 3-----  
raw_data.loc[0:2+1, 'School_Number'] # returns the first 3 rows from column 'School_Number'

# raw_data.loc["subject"] produces an error whereas raw_data.loc[:, "subject"] returns a subset 

0     536
1    1673
2    1919
3     997
Name: School_Number, dtype: int64

## Filtering by DataFrame rows   

We can filter by DataFrames rows in different ways:  
+ Based on variable values i.e. `df[variable_1 == 'A']`   
+ Based on a negated condition i.e. `df[variable_1 != 'A']` or `df[~ condition_A]`     
+ Using `query()` to filter by a single or multiple conditions i.e. `query('variable_1 == 'A' & year == 2002')`      
+ Filtering strings using `.str()` string methods.  

A list of Pandas DataFrame manipulation functions can be found [here](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf).    

In [12]:
#-----example 1----- 
raw_data[raw_data['ID'] <= 3] # filtering by rows  

# raw_data[raw_data.ID <= 3] returns the same output as raw_data[raw_data['ID'] <= 3]  

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [13]:
#-----example 2-----
raw_data[(raw_data.ID <= 3) & (raw_data.Index_of_Educational_Disadvantage != 6)]

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [14]:
#-----example 3-----  
suburbs_list = ['Adelaide', 'Black Forest', 'Woodville']
raw_data[raw_data['Suburb'].isin(suburbs_list)]

# raw_data['Suburb'].isin(suburbs) returns a DataFrame of booleans 

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
4,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678
38,39,548,Black Forest Primary School,Black Forest,5035,Primary Education,6,-34.960682,138.573308
138,139,120,Gilles Street Primary School,Adelaide,5000,Primary Education,4,-34.934490,138.604464
438,439,1881,Sturt Street Community School,Adelaide,5000,Primary Education,5,-34.933817,138.590211
448,449,844,The Grove Education Centre,Woodville,5011,Special Education,4,-34.873425,138.544354


In [15]:
#-----example 4-----  
raw_data[~ raw_data['Suburb'].isin(suburbs_list)] 

# df[~ condition_A] returns the negation of condition_A

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
3,4,997,Adelaide East Education Centre,Paradise,5075,Special Education,6,-34.873042,138.672971
5,6,476,Adelaide North Special School,Munno Para,5115,Special Education,3,-34.663577,138.684948
6,7,1686,Adelaide Secondary School of English,West Croydon,5008,Secondary Education,2,-34.886834,138.565419
...,...,...,...,...,...,...,...,...,...
507,508,501,Yahl Primary School,Yahl,5291,Primary Education,4,-37.879293,140.832014
508,509,999,Yalata Anangu School,Yalata,5690,Aboriginal/Anangu Schools,1,-31.480012,131.843548
509,510,766,Yankalilla Area School,Yankalilla,5203,Primary/Secondary Combined,5,-35.453300,138.333092
510,511,767,Yorketown Area School,Yorketown,5576,Primary/Secondary Combined,4,-35.015376,137.615442


In [16]:
#-----example 5----- 
(raw_data.query('Suburb in @suburbs_list')
         .sort_values(by = ['Suburb', 'School_Name'])
)

# you can also use pd.query to filter conditions
# this enables us to chain Python functions to improve code readability

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
4,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678
138,139,120,Gilles Street Primary School,Adelaide,5000,Primary Education,4,-34.934490,138.604464
438,439,1881,Sturt Street Community School,Adelaide,5000,Primary Education,5,-34.933817,138.590211
38,39,548,Black Forest Primary School,Black Forest,5035,Primary Education,6,-34.960682,138.573308
448,449,844,The Grove Education Centre,Woodville,5011,Special Education,4,-34.873425,138.544354


In [17]:
#-----example 6-----
(raw_data[raw_data['School_Name'].str.startswith('A')]
 .head(3)
)

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
0,1,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743


In [18]:
#-----example 7-----  
(raw_data[raw_data['School_Name'].str.contains('High')]
 .head(3)
)

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
2,3,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743
4,5,768,Adelaide High School,Adelaide,5000,Secondary Education,5,-34.925733,138.586678


In [19]:
#-----example 8-----  
(raw_data[raw_data['School_Name'].str.contains('.*Park.*')]
 .head(3)
)

# str.contains can also evaluate regex

,ID,School_Number,School_Name,Suburb,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage,Latitude,Longitude
1,2,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181
20,21,340,Ascot Park Primary School,Park Holme,5043,Primary Education,3,-34.990774,138.554376
23,24,990,Augusta Park Primary School,Port Augusta,5700,Primary Education,1,-32.495595,137.784949


## Selecting DataFrame columns   

We can select DataFrame columns in different ways:  
+ Based on column names i.e. `df[['variable_1', 'variable_2']]`  
+ By filtering column names using regular expressions i.e. `df.filter(regex = regex)`   

In [20]:
#-----example 1-----  
(raw_data[['Post_Code', 'Type_of_Schooling', 'Index_of_Educational_Disadvantage']]
.head(3)
)

,Post_Code,Type_of_Schooling,Index_of_Educational_Disadvantage
0,5159,Primary Education,6
1,5159,Secondary Education,6
2,5000,Secondary Education,7


In [21]:
#-----example 2-----
print(type(raw_data.Suburb)) 
print(type(raw_data['Suburb']))  
print(type(raw_data[['Suburb']]))

# df[['variable_1']] returns a DataFrame but df['variable_1'] or df.variable_1 returns a Series  

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [22]:
#-----example 3-----
contains_school = df.filter(regex = '.*(S|s)chool.*')
contains_school.head(3)

# in Python, filter returns columns via regex matching

,School_Number,School_Name,Type_of_Schooling
0,536,Aberfoyle Hub R-7 School,Primary Education
1,1673,Aberfoyle Park High School,Secondary Education
2,1919,Adelaide Botanic High School,Secondary Education


To rename a column, we can use a dictionary inside the rename function i.e. `df.rename(columns = {'old_name' : 'new_name'})` or by applying string methods on `df.columns`.        

In [23]:
#-----example 1-----  
raw_data = raw_data.rename(columns = {'Index_of_Educational_Disadvantage' : 'edu_disadvantage_index',
                                      'Type_of_Schooling' : 'schooling_type'})  

raw_data.columns

Index(['ID', 'School_Number', 'School_Name', 'Suburb', 'Post_Code',
       'schooling_type', 'edu_disadvantage_index', 'Latitude', 'Longitude'],
      dtype='object')

In [24]:
#-----example 2-----
raw_data.columns = raw_data.columns.str.lower()  

raw_data.columns

Index(['id', 'school_number', 'school_name', 'suburb', 'post_code',
       'schooling_type', 'edu_disadvantage_index', 'latitude', 'longitude'],
      dtype='object')

To delete a single or multiple columns, we can use the drop function i.e. `df.drop(['variable_1', 'variable_2'])`.  

In [25]:
#-----example 3-----  
raw_data = raw_data.drop(columns = ['id'])

raw_data.columns

Index(['school_number', 'school_name', 'suburb', 'post_code', 'schooling_type',
       'edu_disadvantage_index', 'latitude', 'longitude'],
      dtype='object')

## Transforming data    

Data transformations are usually required to create new variables of interest. We can transform columns into new columns in different ways:  

+ Be applying a pre-existing function on a single or multiple columns   
+ By applying a function along an axis of the DataFrame using `apply()` i.e. `df.apply(lambda x: x.method if condition == 1 else x)`     
+ By applying a function elementwise across a DataFrame i.e. `applymap()`. 
    + Note that `applymap()` has been optimized may work much faster than `apply()` in certain cases.   
+ By substituting individual values in a Series with another value i.e. `map()`  
    + Note that `map()` accepts a dictionary, Series or function. Values that are not found in the dict are converted to NaN.  

**Note:** A [lambda function](https://realpython.com/python-lambda/) is a function that does not necessarily need to be assigned a name.  

In [26]:
#-----example 1-----  
raw_data['id'] = np.arange(1, raw_data.shape[0] + 1, 1) 
raw_data.head(3)

# create new Series using pre-defined np function

,school_number,school_name,suburb,post_code,schooling_type,edu_disadvantage_index,latitude,longitude,id
0,536,Aberfoyle Hub R-7 School,Aberfoyle Park,5159,Primary Education,6,-35.078139,138.595142,1
1,1673,Aberfoyle Park High School,Aberfoyle Park,5159,Secondary Education,6,-35.074007,138.594181,2
2,1919,Adelaide Botanic High School,Adelaide,5000,Secondary Education,7,-34.917055,138.606743,3


In [27]:
#-----example 2.1-----
df_2 = pd.DataFrame(np.arange(1, 10+1).reshape(5,2),
                    columns = ['var1','var2'])

df_2 # create df_2

,var1,var2
0,1,2
1,3,4
2,5,6
3,7,8
4,9,10


In [28]:
#-----example 2.2-----  
df_2.apply(lambda x: x ** 2)  

# by default, a lambda is applied across the whole DataFrame

,var1,var2
0,1,4
1,9,16
2,25,36
3,49,64
4,81,100


In [29]:
#-----example 2.3-----
df_2['var1_squared'] = df_2['var1'].apply(lambda x: x ** 2) 
df_2

# we can also specify individual columns to transform

,var1,var2,var1_squared
0,1,2,1
1,3,4,9
2,5,6,25
3,7,8,49
4,9,10,81


**Note:** If we want to add columns `var1` and `var2` together, we can either directly specify this transformation or define a new function and specify which axis we would like it to apply across.       

In [30]:
#-----example 2.4-----
df_2['vars_combined'] = df_2['var1'] + df_2['var2'] 
df_2

,var1,var2,var1_squared,vars_combined
0,1,2,1,3
1,3,4,9,7
2,5,6,25,11
3,7,8,49,15
4,9,10,81,19


In [31]:
#-----example 2.5-----
df_2['vars_combined'] = df_2.apply(lambda x: x['var1'] + x['var2'], axis=1) 
df_2

,var1,var2,var1_squared,vars_combined
0,1,2,1,3
1,3,4,9,7
2,5,6,25,11
3,7,8,49,15
4,9,10,81,19


**Note:** The output created by `df.apply(lambda function, axis=1, result_type='expand')` is similar to the output created using `mutate_all()` in R. 

In [32]:
#-----example 2.6-----  
def cumulative_add(row): 
    return [row['var1'] + 1, row['var2'] + 2] 

print(df_2.apply(cumulative_add, axis = 1))
print(df_2.apply(cumulative_add, axis = 1, result_type = 'expand'))

# we can assign an unnamed lambda function or named function inside apply()

0      [2, 4]
1      [4, 6]
2      [6, 8]
3     [8, 10]
4    [10, 12]
dtype: object
    0   1
0   2   4
1   4   6
2   6   8
3   8  10
4  10  12


In [33]:
#-----example 2.7----- 

## Aggregating data

## Reshaping DataFrames 

## Joining datasets  